In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

import yfinance as yf
from yahooquery import Ticker
from yahooquery import Screener
# import plotly.graph_objects as go

from sklearn.model_selection import TimeSeriesSplit  # you have everything done for you
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pandas.core.window import Rolling
import statsmodels.api as sm 

from pmdarima.arima import auto_arima
from pmdarima import pipeline
from pmdarima import model_selection
from pmdarima import preprocessing as ppc
from pmdarima import arima